# DevOps Engineer Interview Project
**Author : Felix Yuzhou Sun**

## 0. Introduction

In the context of financial analysis, accurately extracting Earnings Per Share (EPS) data from regulatory filings is essential for investors, analysts, and financial professionals. This task, often complicated by the diverse formats and structures of financial documents, requires a robust and adaptable parsing solution.

This project involves developing a parser to extract EPS data from SEC EDGAR filings in HTML format, as part of an assessment for a DevOps Engineer Interview at Trexquant Investment LP. The primary objective is to create a versatile parser that can handle various filing formats, systematically extract the latest quarterly EPS for each company, and present this data in a structured CSV format.

The development and testing of this parser are grounded in the analysis of 50 provided HTML files, focusing on identifying common patterns, handling variations, and ensuring the parser's adaptability to unseen document formats. The outcome of this project will be a reliable tool capable of facilitating informed financial decision-making through accurate data extraction.

## 1. Understanding the data

To build a robust parser, we will meticulously analyze the 50 training filings, identifying the common patterns and locations where EPS data is typically reported. This analysis will encompass recognizing keywords, table structures, and contextual cues that signal the presence of relevant information. We will also pay close attention to potential inconsistencies and edge cases, ensuring our parser's accuracy and adaptability across different EDGAR filings.

In [ ]:
# Set Up Environment
%pip install beautifulsoup4 pandas
%pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [1]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import defaultdict

In [6]:
# Check the html data
current_directory = os.getcwd()
data_directory  = 'Training_Filings'
data_path = os.path.join(current_directory, data_directory)
print(f"Current working directory: {current_directory}")
print(data_path)

# Verify the data size
html_files = [f for f in os.listdir(data_path) if f.endswith('.html')]
# Total number of files
total_files = len(html_files)
print(f"Found {total_files} HTML files.")

Current working directory: /Users/felixsun/Documents/GitHub/EPS_Paser
/Users/felixsun/Documents/GitHub/EPS_Paser/Training_Filings
Found 50 HTML files.


### A. Check the data structure: Tag Analysis

In [ ]:
%%time
# Dictionary to hold the number of files in which each tag appears
tag_file_count = defaultdict(int)

# Iterate through all files and count tags
for file in html_files:
    filepath = os.path.join(data_path, file)
    
    with open(filepath, 'r', encoding='utf-8') as html_file:
        soup = BeautifulSoup(html_file, 'html.parser')
        
        # Extract unique tags from the current file
        unique_tags_in_file = set(tag.name for tag in soup.find_all(True))
        
        # Update the count of each tag, but only once per file
        for tag in unique_tags_in_file:
            tag_file_count[tag] += 1

# Calculate the percentage of files in which each tag appears
tag_percentage = {tag: (count / total_files) * 100 for tag, count in tag_file_count.items()}

# Sort the tags by percentage in descending order
sorted_tags = sorted(tag_percentage.items(), key=lambda item: item[1], reverse=True)

# Print results
print(f"{'Tag':<15}{'Files':<10}{'Percentage':<10}")
for tag, percentage in sorted_tags:
    print(f"{tag:<15}{tag_file_count[tag]:<10}{percentage:<10.2f}%")

#### Tag analysis results

| Tag 1         | Files | Percentage | Tag 2         | Files | Percentage |
|---------------|-------|------------|---------------|-------|------------|
| body          | 50    | 100.00%    | br            | 46    | 92.00%     |
| document      | 50    | 100.00%    | hr            | 42    | 84.00%     |
| filename      | 50    | 100.00%    | sup           | 38    | 76.00%     |
| font          | 50    | 100.00%    | img           | 36    | 72.00%     |
| html          | 50    | 100.00%    | a             | 26    | 52.00%     |
| sequence      | 50    | 100.00%    | p             | 23    | 46.00%     |
| table         | 50    | 100.00%    | b             | 14    | 28.00%     |
| td            | 50    | 100.00%    | u             | 13    | 26.00%     |
| text          | 50    | 100.00%    | i             | 8     | 16.00%     |
| tr            | 50    | 100.00%    | ul            | 4     | 8.00%      |
| type          | 50    | 100.00%    | li            | 4     | 8.00%      |
| title         | 49    | 98.00%     | center        | 4     | 8.00%      |
| description   | 49    | 98.00%     | h1            | 3     | 6.00%      |
| div           | 49    | 98.00%     | h2            | 3     | 6.00%      |
| em            | 1     | 2.00%      | meta          | 2     | 4.00%      |
| strike        | 1     | 2.00%      | strong        | 1     | 2.00%      |



#### Insights from the Tag Analysis


1. **Core Tags (`table`, `td`, `tr`)**:
   - **High Occurrence (100%)**: Tags like `<table>`, `<td>`, `<tr>`, and related tags appear in every document. This indicates that EPS data, which is typically presented in tabular format, is consistently structured within tables. Therefore, the parser should primarily focus on searching within `<table>` tags to locate EPS values.

2. **Document Structure (`html`, `body`, `title`)**:
   - **High Occurrence (100%)**: Tags such as `<html>`, `<body>`, and `<title>` are present in every file, indicating that all documents follow a well-defined HTML structure. The `<title>` tag, which often contains the document's title, might give context (e.g., the company name or report type) that could help identify relevant sections of the document.

3. **Financial Data Tags (`sup`, `sub`)**:
   - **Moderate Occurrence (76%, 84%)**: Tags like `<sup>` (often used for superscripts like footnotes) might be associated with small notations beside financial data, including EPS figures. These tags are important for handling any associated footnotes or references that could alter the interpretation of the EPS data (e.g., adjustments for extraordinary items).
   
4. **Formatting Tags (`font`, `br`, `hr`, `b`)**:
   - **High to Moderate Occurrence**: Tags like `<font>`, `<br>`, and `<b>` indicate that the documents use various formatting styles to highlight or structure the data. Bold tags (`<b>`) often emphasize key financial metrics, including EPS.

### B. Recognize "Earnings Per Share" patterms

"Earnings Per Share" (EPS) might appear in various forms, including abbreviations, acronyms, or even partially spaced out versions like "E...P...S...". We need to capture not just standard forms like "EPS" or "Earnings Per Share," but also non-standard forms like "E... P... S..." or "earnings p/share." 

#### High-Level Methodology:

1. **Isolate Relevant Sections**:
   - **Goal**: Focus on parts of the document that are likely to contain EPS-related information, primarily the tables.
   - **Method**: Extract all `<table>` elements from each HTML file since these sections are structured and likely contain financial data.

2. **Tokenize Content**:
   - **Goal**: Break down the table content into manageable pieces (tokens) to identify relevant information.
   - **Method**: Treat each cell within a table (`<td>` or `<th>`) as a token. These tokens will contain words or phrases that can be analyzed individually.

3. **Pattern Matching**:
   - **Goal**: Identify tokens that are likely to be related to EPS by searching for specific keywords or phrases.
   - **Method**: Look for the presence of certain key components within each token, such as:
     - **"earn"**: Captures "earnings," "earn," "earning," etc.
     - **"per"**: Relevant for identifying "per share."
     - **"share"**: Specifically targets mentions of shares.
     - **"loss"**: Captures contexts like "net income (loss)" or "earnings (loss) per share."

4. **Contextual Analysis**:
   - **Goal**: Ensure the identified tokens are relevant to EPS and not unrelated financial terms (like "expenses").
   - **Method**: Consider the context in which these tokens appear. This involves checking the neighboring tokens or phrases to confirm they relate to EPS (e.g., "net income (loss)" or "GAAP earnings per share").

5. **Extraction and Aggregation**:
   - **Goal**: Extract the relevant phrases or terms and aggregate them across all documents.
   - **Method**: Combine tokens that are identified as related to EPS into complete phrases, and track their occurrences across all files.

6. **Generalization**:
   - **Goal**: Develop a robust approach that can handle variations in how EPS is presented across different documents.
   - **Method**: Use the patterns identified to create generalized rules that can be applied to other datasets.




         

#### Exploratory Text Mining

In [12]:
import os
from bs4 import BeautifulSoup
from spacy.matcher import Matcher
from collections import defaultdict
from tqdm import tqdm


**We start with finding out tokens with "per" and "share", then try to narrow down the patterm.**

In [32]:
# Function to extract tables from HTML content
def extract_tables(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.find_all('table')

# Function to tokenize content from a table
def tokenize_table(table):
    tokens = []
    for row in table.find_all('tr'):
        for cell in row.find_all(['td', 'th']):
            tokens.append(cell.get_text(strip=True).lower())
    return tokens

# Simplified function to match tokens that contain "per" and "share"
def match_eps_patterns(tokens):
    eps_related_phrases = []
    for token in tokens:
        if 'per' in token and 'share' in token:  # Only check for "per" and "share"
            eps_related_phrases.append(token)
    return eps_related_phrases

# Function to filter out irrelevant phrases
def filter_relevant_phrases(phrases):
    relevant_phrases = []
    for phrase in phrases:
        # Additional filtering logic to remove irrelevant terms
        if 'expenses' not in phrase and 'expectations' not in phrase:
            relevant_phrases.append(phrase)
    return relevant_phrases

# Function to aggregate EPS-related data across all HTML files
def aggregate_eps_data(html_files):
    eps_data = defaultdict(set)
    files_with_matches = 0  # Counter for files with at least one match

    for file in tqdm(html_files, desc="Processing files"):
        filepath = os.path.join(data_path, file)
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
            tables = extract_tables(content)
            file_has_match = False  # Track if this file has at least one match

            for table in tables:
                tokens = tokenize_table(table)
                eps_phrases = match_eps_patterns(tokens)
                relevant_phrases = filter_relevant_phrases(eps_phrases)

                if relevant_phrases:
                    file_has_match = True
                    eps_data[file].update(relevant_phrases)

            if file_has_match:
                files_with_matches += 1  # Increment counter if a match is found

    return eps_data, files_with_matches

In [33]:
# Run the aggregation function and print the results
eps_data, files_with_matches = aggregate_eps_data(html_files)

# Print the number of files with at least one match
total_files = len(html_files)
print(f"\nNumber of files with at least one match: {files_with_matches} out of {total_files}")


Processing files: 100%|██████████| 50/50 [00:09<00:00,  5.17it/s]


Number of files with at least one match: 50 out of 50


In [31]:
# Print the set of EPS candidates for each file
for file, eps_set in eps_data.items():
    print(f"\nFile: {file}")
    print("EPS-related Phrases:")
    for phrase in eps_set:
        print(f"  - {phrase}")


File: 0001008654-20-000048.html
EPS-related Phrases:
  - (in millions, except per share data)
  - diluted income (loss) per share
  - net income (loss) per common share:
  - net income (loss) per share (diluted)
  - basic income (loss) per share

File: 0001140361-20-010070.html
EPS-related Phrases:
  - net earnings per common share:
  - net earnings per common share - basic
  - net earnings per common share - diluted
  - gaap diluted earnings per share from continuing operations
  - diluted earnings per share from continuing operations
  - non-gaap diluted earnings per share from continuing operations

File: 0001193125-20-124288.html
EPS-related Phrases:
  - net income per common share
  - per share data:
  - (in thousands, except per share data)
  - cash dividends declared per common share

File: 0001299709-20-000078.html
EPS-related Phrases:
  - (dollars in thousands, except per share data)
  - tangible book value per common share (non-gaap)
  - common shares outstanding at end of p